In [12]:
import requests, sys, json, random, time
from confluent_kafka import Consumer, Producer

In [13]:
def get_req(endpoint):
    response = requests.get(endpoint)
    
    print('STATUS:', response.status_code)
    return response.json()

In [14]:
def post_req(endpoint, body):
    response = requests.post(endpoint, json=body)
    
    print('STATUS:', response.status_code)
    return response.json()

### CREATE A NEW KAFKA TOPIC (VIA BACKEND API)

In [15]:
post_req('http://localhost:3003/kafka/create', {
    'name': 'test_topic',
    'num_partitions': 2
})

STATUS: 201


{'topic_name': 'test_topic', 'num_partitions': 2}

### CREATE A NEW CASSANDRA TABLE (VIA BACKEND API)

In [5]:
post_req('http://localhost:3003/cassandra/create', {
    'domain': 'foobar.test_table',
    'columns': {
        'timestamp': 'int',
        'open': 'float',
        'close': 'float',
        'high': 'float',
        'low': 'float',
        'volume': 'int',
    },
    'indexing': [
        'timestamp'
    ]
})

STATUS: 201


### PUSH TEST DATA TO A KAFKA TOPIC

In [18]:
KAFKA_SERVERS = 'localhost:11001,localhost:11002'
VERBOSE = True

class create_producer:

    # ON LOAD, CREATE KAFKA PRODUCER
    def __init__(self):
        self.kafka_client = Producer({
            'bootstrap.servers': KAFKA_SERVERS,
        })

    # MAKE SURE KAFKA CONNECTION IS OK
    def connected(self):
        try:
            metadata = self.kafka_client.list_topics(timeout=2)
            print('SUCCESSFULLY CONNECTED TO KAFKA')
            return True
        except:
            print(f'COULD NOT CONNECT WITH KAFKA SERVER ({KAFKA_SERVERS})')
            return False

    # ON CONSUMER CALLBACK, DO..
    def ack_callback(self, error, message):
        if error:
            print('ACK ERROR', error)
        else:
            if VERBOSE: print(f'MESSAGE PUSHED')

    # PUSH MESSAGE TO A KAFK TOPIC
    def push_msg(self, topic_name: str, json_data: dict):

        # PUSH MESSAGE TO KAFKA TOPIC
        self.kafka_client.produce(
            topic_name, 
            value=json.dumps(json_data).encode('utf-8'),
            on_delivery=self.ack_callback,
        )

        # ASYNCRONOUSLY AWAIT CONSUMER ACK BEFORE SENDING NEXT MSG
        self.kafka_client.poll(1)
        # self.kafka_client.flush()

kafka_producer = create_producer()

In [25]:
kafka_producer.push_msg('input_data', {
    'timestamp': int(time.time()),
    'open': round(random.uniform(5000, 30000), 3),
    'close': round(random.uniform(5000, 30000), 3),
    'high': round(random.uniform(5000, 30000), 3),
    'low': round(random.uniform(5000, 30000), 3),
    'volume': random.randint(0, 100000)
})

MESSAGE PUSHED
